In [16]:
import numpy as np
import pandas as pd
from scipy.special import softmax


In [76]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig

from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score


In [77]:
import torch

print(torch.__version__)

torch.cuda.is_available()

2.2.1


False

## Let's do the EDA.

In [4]:
df = pd.read_csv("youtube_comments.csv", delimiter=',')
df_comments= df["Comment"]

In [71]:
text  = df_comments[:]
text

0                                      USHER IS A LEGEND 🔥
1                                                    Yeah!
2                                           Let Him Cook!!
3                                    Ok so I ate a raccoon
4        All these star performers came together tonigh...
                               ...                        
38296                         Fireeeee asfff usher Ateeee😭
38297                                                first
38298                                          yuppp usher
38299                                          Legendary 🔥
38300                                              Usher??
Name: Comment, Length: 38301, dtype: object

In [72]:
modified_text = []
for i in range(len(text)): 
    # Check if the element is NaN
    if isinstance(text[i], str):
        # Initialize an empty list to store modified words
        modified_words = []
        # Split the text into words
        for word in text[i].split(' '):
            if ' ' not in word and ('@@' in word or word.startswith('http')):
                continue
            elif '@@' in word:
                word = word.replace('@@', '@user')
            elif word.startswith('http'):
                word = 'http'

            # Keep the original word if it doesn't match any conditions
            modified_words.append(word)
        # Check if the list of modified words is empty
        if modified_words:
            # Join the modified words into a single string and append it to modified_text
            modified_text.append(' '.join(modified_words))


In [73]:
len(modified_text)

38072

## load the model


In [11]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
# config = AutoConfig.from_pretrained(MODEL)
# model = AutoModelForSequenceClassification.from_pretrained(MODEL)

config.json: 100%|██████████| 747/747 [00:00<00:00, 1.51MB/s]
vocab.json: 100%|██████████| 899k/899k [00:03<00:00, 299kB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 512kB/s]
special_tokens_map.json: 100%|██████████| 150/150 [00:00<00:00, 418kB/s]


model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [12]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

pytorch_model.bin: 100%|██████████| 499M/499M [18:13<00:00, 456kB/s] 


## split the data


In [81]:
labels = ['Negative', 'Neutral', 'Positive']
# # Split data into training and validation sets (e.g., 80% for training, 20% for validation)
# X_train, X_val, y_train, y_val = train_test_split(modified_text, labels, test_size=0.2, random_state=42)


In [68]:


# Initialize variables to store cumulative sentiment scores
total_scores = np.zeros(len(labels))
count = 0
# Loop through each text in your dataset
for text in modified_text:
    # Encode the comment
    encoded_comment = tokenizer(text, return_tensors='pt')
    
    # Analyze sentiment using the model
    output = model(**encoded_comment)
    
    # Extract sentiment scores
    scores = output[0][0].detach().numpy()
    
    # Apply softmax to convert scores to probabilities
    scores = softmax(scores)
    
    # Accumulate sentiment scores
    total_scores += scores
    count = count + 1
    

# Calculate average sentiment scores
print(count)
average_scores = total_scores / len(modified_text)

972


In [69]:
print(total_scores)

[212.89004262 336.22087285 422.88908344]


In [70]:
# Print average sentiment scores
for label, score in zip(labels, average_scores):
    print(f"{label}: {score:.4f}")

Negative: 0.2190
Neutral: 0.3459
Positive: 0.4351
